In [59]:
import yaml
import numpy as np
import numpy.random as rnd
import time

In [60]:
class host:
    
    def __init__(self, info):
        self.__dict__ = info
        
        for key, item in self.__dict__.items():
            self.__dict__[key] = np.array(item) if type(item) is list else item
        
        self.length = 10
        self.objLba = self.max_objLba
    
    def gen_req(self):
        self.objLba += self.length
        self.gen_req_type()
        self.gen_req_length()
        self.gen_req_wait_timestamp()
        if self.new_objLba() or self.objLba + self.length >= self.max_objLba:
            self.gen_objLba()
            while self.objLba + self.length >= self.max_objLba:
                self.gen_objLba()
        
    def new_objLba(self):
        return rnd.choice([0,1], p=self.new_offset_prob)
        
    def gen_req_type(self):
        self.tp = rnd.choice(['READ', 'WRITE'], p=self.rw_prob)
        self.opCode = '0x128001' if self.tp == 'READ' else '0x13218000'
        self.result = 0
            
    def gen_req_length(self):
        self.length = rnd.choice(self.req_len, p=self.req_len_prob)
        
    def gen_req_wait_timestamp(self):
        self.timestamp = rnd.choice(self.req_wait, p=self.req_wait_prob)
        self.timepoint = 1
        self.last_time_nano = self.timestamp
    
    def gen_objLba(self):
        tmp = rnd.choice([0,1], p=self.mem_prob)
        self.objLba = rnd.randint(int(self.max_objLba * self.mem_split[tmp])) +\
                                                int(self.max_objLba * self.mem_split[0]) * tmp
        self.volumeId = '0x6e'
        self.objId = '0x6e'
        
    def send_req(self, columns):
        self.gen_req()
        return [self.__dict__[key] for key in columns]
    

In [61]:
class host_handler:
    
    def __init__(self, cfg_file):
        cfg = [i for i in yaml.safe_load_all(cfg_file)]
        self.__dict__ = cfg[0]
        self.out = open(self.output, 'w')
        del cfg[0]
        self.active_hosts = []
        self.host_prob = []
        for i in cfg:
            i['max_objLba'] = self.max_objLba
            tmp_host = host(i)
            self.active_hosts.append(tmp_host)
            self.host_prob.append(tmp_host.host_prob)
        self.host_prob = np.array(self.host_prob)
        self.out.write(self.pattern.format(*self.columns) + '\n')
        
    def gen_trace(self):
        curr_time = time.time_ns()
        for i in range(self.max_reqs):
            req = rnd.choice(self.active_hosts, p=self.host_prob).send_req(self.columns)
            if 'timestamp' in self.columns:
                ind = self.columns.index('timestamp')
                curr_time += req[ind]
                req[ind] = time.strftime("%a %b  %d %X %Y ",time.localtime(curr_time/(10**9)))
                if 'last_time_nano' in self.columns:
                    ind = self.columns.index('last_time_nano')
                    req[ind] = curr_time
            self.out.write(self.pattern.format(*req) + '\n')
        self.out.close()

In [62]:
cfg = open('cfg.yaml', 'r')
t = host_handler(cfg)
t.gen_trace()